In [ ]:
import tensorflow as tf
import cv2
import asyncio
import numpy as np
from ultralytics import YOLO

# Load your YOLO models
hard_hat_model = YOLO("C:/Users/kvsbb/Downloads/forko.pt")
forklift_model = YOLO("C:/Users/kvsbb/Downloads/head.pt")

# Load MoveNet model
model_dir = "C:/Users/kvsbb/movenet"
loaded_model = tf.saved_model.load(model_dir)
infer = loaded_model.signatures["serving_default"]

# Define edges for MoveNet
EDGES = {
    (0, 1): 'm', (0, 2): 'c', (1, 3): 'm', (2, 4): 'c',
    (0, 5): 'm', (0, 6): 'c', (5, 7): 'm', (7, 9): 'm',
    (6, 8): 'c', (8, 10): 'c', (5, 6): 'y', (5, 11): 'm',
    (6, 12): 'c', (11, 12): 'y', (11, 13): 'm', (13, 15): 'm',
    (12, 14): 'c', (14, 16): 'c'
}

# Masking function to define region of interest
def apply_mask(frame, mask_region):
    return cv2.rectangle(frame, mask_region[0], mask_region[1], (0, 0, 0), -1)

# Async function for YOLO inference
async def run_yolo_inference(model, frame):
    return model(frame)

# MoveNet inference
def run_movenet_inference(frame):
    img = frame.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = tf.image.resize_with_pad(tf.convert_to_tensor(img), 384, 640)
    input_img = tf.cast(img, dtype=tf.int32)
    input_img = tf.expand_dims(input_img, axis=0)
    results = infer(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6, 17, 3))
    return keypoints_with_scores

# Draw MoveNet keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Async function to process video
async def process_video_async():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize for video compression
        compressed_frame = cv2.resize(frame, (1280, 720))  # From 2K to 720p

        # Apply masking
        mask_region = [(50, 50), (500, 500)]  # Define the region for masking
        masked_frame = apply_mask(compressed_frame, mask_region)

        # YOLO inference
        yolo_task1 = asyncio.create_task(run_yolo_inference(hard_hat_model, masked_frame))
        yolo_task2 = asyncio.create_task(run_yolo_inference(forklift_model, masked_frame))

        # MoveNet inference
        keypoints_with_scores = run_movenet_inference(masked_frame)

        # Await YOLO results
        hard_hat_result = await yolo_task1
        forklift_result = await yolo_task2

        # Flag for detecting presence of hard hats and forklifts
        hard_hat_detected = False
        forklift_detected = False

        # Display YOLO detections for hard hats
        for result in hard_hat_result:
            boxes = result.boxes  # Access boxes object
            if len(boxes) > 0:
                hard_hat_detected = True
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Extract box coordinates
                    cv2.rectangle(masked_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue box for hard hats

        # Display YOLO detections for forklifts
        for result in forklift_result:
            boxes = result.boxes  # Access boxes object
            if len(boxes) > 0:
                forklift_detected = True
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Extract box coordinates
                    cv2.rectangle(masked_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box for forklifts

        # Print messages if no hard hats or forklifts are detected
        if not hard_hat_detected:
            print("No hard hats detected.")
        if not forklift_detected:
            print("No forklifts detected.")

        # Draw MoveNet keypoints and connections
        loop_through_people(masked_frame, keypoints_with_scores, EDGES, 0.1)

        # Show the processed frame
        cv2.imshow('Fused Detection', masked_frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Event loop management
try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # No event loop is running
    loop = None

# Execute based on whether an event loop is already running
if loop and loop.is_running():
    # If an event loop is running, await the async function directly
    await process_video_async()
else:
    # If no event loop is running, start a new one using asyncio.run()
    asyncio.run(process_video_async())
#WORKS

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO

# Load your YOLO models
hard_hat_model = YOLO("C:/Users/kvsbb/Downloads/forko.pt")
forklift_model = YOLO("C:/Users/kvsbb/Downloads/head.pt")

# Load MoveNet model
model_dir = "C:/Users/kvsbb/movenet"
loaded_model = tf.saved_model.load(model_dir)
infer = loaded_model.signatures["serving_default"]

# Define edges for MoveNet
EDGES = {
    (0, 1): 'm', (0, 2): 'c', (1, 3): 'm', (2, 4): 'c',
    (0, 5): 'm', (0, 6): 'c', (5, 7): 'm', (7, 9): 'm',
    (6, 8): 'c', (8, 10): 'c', (5, 6): 'y', (5, 11): 'm',
    (6, 12): 'c', (11, 12): 'y', (11, 13): 'm', (13, 15): 'm',
    (12, 14): 'c', (14, 16): 'c'
}

# Masking function to define region of interest
def apply_mask(frame, mask_region):
    return cv2.rectangle(frame, mask_region[0], mask_region[1], (0, 0, 0), -1)

# MoveNet inference
def run_movenet_inference(frame):
    img = frame.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = tf.image.resize_with_pad(tf.convert_to_tensor(img), 384, 640)
    input_img = tf.cast(img, dtype=tf.int32)
    input_img = tf.expand_dims(input_img, axis=0)
    results = infer(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6, 17, 3))
    return keypoints_with_scores

# Draw MoveNet keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, _ = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Function to process video
def process_video():
    cap = cv2.VideoCapture(0)
    
    frame_count = 0
    frame_skip = 2  # Skip every second frame for performance

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames to reduce computational load
        
        # Resize the frame to 640p for faster processing
        compressed_frame = cv2.resize(frame, (640, 360))

        # Apply masking
        mask_region = [(50, 50), (500, 500)]  # Define the region for masking
        masked_frame = apply_mask(compressed_frame, mask_region)

        # Run inferences in parallel using threads
        with ThreadPoolExecutor(max_workers=3) as executor:
            yolo_task1 = executor.submit(hard_hat_model.predict, masked_frame)
            yolo_task2 = executor.submit(forklift_model.predict, masked_frame)
            movenet_result = run_movenet_inference(masked_frame)  # Running MoveNet synchronously

            # Collect YOLO results from threads
            hard_hat_result = yolo_task1.result()
            forklift_result = yolo_task2.result()

        # Flag for detecting presence of hard hats and forklifts
        hard_hat_detected = False
        forklift_detected = False

        # Display YOLO detections for hard hats
        for result in hard_hat_result:
            boxes = result.boxes  # Access boxes object
            if len(boxes) > 0:
                hard_hat_detected = True
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Extract box coordinates
                    cv2.rectangle(masked_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue box for hard hats

        # Display YOLO detections for forklifts
        for result in forklift_result:
            boxes = result.boxes  # Access boxes object
            if len(boxes) > 0:
                forklift_detected = True
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Extract box coordinates
                    cv2.rectangle(masked_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box for forklifts

        # Print messages if no hard hats or forklifts are detected
        if not hard_hat_detected:
            print("No hard hats detected.")
        if not forklift_detected:
            print("No forklifts detected.")

        # Draw MoveNet keypoints and connections
        loop_through_people(masked_frame, movenet_result, EDGES, 0.1)

        # Show the processed frame
        cv2.imshow('Fused Detection', masked_frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to run the video processing
if __name__ == "__main__":
    process_video()
#AlsoCheckThis



0: 384x640 (no detections), 741.3ms
Speed: 32.2ms preprocess, 741.3ms inference, 15.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 886.2ms
Speed: 15.6ms preprocess, 886.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 875.5ms
Speed: 8.0ms preprocess, 875.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 1083.0ms
Speed: 17.0ms preprocess, 1083.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 792.0ms
Speed: 7.4ms preprocess, 792.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 1064.5ms
Speed: 5.5ms preprocess, 1064.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 809.1ms
Speed:

No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 578.2ms
Speed: 4.2ms preprocess, 578.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 755.0ms
Speed: 8.1ms preprocess, 755.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 652.4ms
Speed: 15.0ms preprocess, 652.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 788.6ms
Speed: 4.9ms preprocess, 788.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 384x640 (no detections), 591.2ms
Speed: 6.3ms preprocess, 591.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 727.2ms
Speed: 0.0ms preprocess, 727.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.
No forklifts detected.


0: 

No hard hats detected.


0: 384x640 (no detections), 574.2ms
Speed: 4.2ms preprocess, 574.2ms inference, 13.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 NO-Hardhat, 798.6ms
Speed: 3.0ms preprocess, 798.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.


0: 384x640 (no detections), 599.6ms
Speed: 0.0ms preprocess, 599.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 2 NO-Hardhats, 727.8ms
Speed: 15.6ms preprocess, 727.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.


0: 384x640 (no detections), 643.1ms
Speed: 11.2ms preprocess, 643.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 NO-Hardhat, 857.6ms
Speed: 9.6ms preprocess, 857.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.


0: 384x640 1 person, 661.6ms
Speed: 6.6ms preprocess, 661.6ms inference, 3.3ms postprocess per image 

0: 384x640 1 NO-Hardhat, 697.1ms
Speed: 4.9ms preprocess, 697.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0: 384x640 1 person, 566.0ms
Speed: 13.5ms preprocess, 566.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 NO-Hardhat, 692.2ms
Speed: 15.7ms preprocess, 692.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


0: 384x640 (no detections), 627.0ms
Speed: 0.0ms preprocess, 627.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 NO-Hardhat, 743.0ms
Speed: 9.0ms preprocess, 743.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.


0: 384x640 (no detections), 623.4ms
Speed: 6.0ms preprocess, 623.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 NO-Hardhat, 721.4ms
Speed: 3.2ms preprocess, 721.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
No hard hats detected.


0: 384x640 (no detections